# 手把手教你BERT中文文本分类-第一篇

作者：你们大卫

时间：2020年4月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

## 写在开头

### 感言

BERT 模型自2018年发布以来，它和它的衍生品几乎在NLP圈一统天下。关于BERT模型的原理我就不系统介绍了，各位自行去读论文和看博客分析。

这篇Notebook的起因，是我在使用 PyTorch 和 Keras 学习和实验 BERT 系列模型（包括RoBERTa、ALBERT等）的时候，看着 CSDN、Github、知乎 等论坛上五花八门的封装代码头疼——尤其是中文任务的代码，大多数代码作者估计都是知其然而不知其所以然，用得上用不上的统统封装，又臭又长、缺失注释、结构混乱，对初学者极其不友好，看得想吐。

此外，这些代码还有一些版本问题，就是基于 PyTorch 的 BERT 框架已经进化为了 [transformers](https://github.com/huggingface/transformers)，而CSDN、知乎、Github上很多项目还是基于**旧版** `pytorch-pretrained-bert` 框架，参考价值有限。

新框架支持 ALBERT、RoBERTa 等新模型的调用，模型功能上也有所更新，更何况我本人在技术上爱用新不爱用旧，所以对很多基于旧版本框架的中文任务参考代码很是头疼。

基于这些原因，我在进行了一段时间的实验以后，决定基于 PyTorch 和 [transformers](https://github.com/huggingface/transformers) 框架写一篇**新手友好**的 BERT 实战教程。

> 注意，这篇教程可以说是 Baseline，里面缺少了很多工程实用处理，比如：
>
>- BERT.Config封装；
>- 文本预处理；
>- 半精度处理；
>- 数据读写改进；
>- 显存监控；
>- TorchSnooper；
>- 自定义损失函数；
>- 自定义网络结构；
>
>计划是以后在另一篇完善，本篇任务仅仅是简洁易懂地说明 BERT 和 [transformers](https://github.com/huggingface/transformers) 框架的使用。

### 基础要求

- Python基础；
- 你有一定的 PyTorch 使用经验；
- 你对 NLP 有一些经验；
- 你对 BERT 和 Tranformer 的理论和结构有一定的了解；
- 你想使用 BERT 系列模型执行一些 NLP 任务，比如 NER、文本分类等。

### Transformers框架简介

🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides state-of-the-art general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet, CTRL...) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch.

Github地址：https://github.com/huggingface/transformers

文档地址：https://huggingface.co/transformers/index.html

transformers 框架横跨 TF2.0 和 PyTorch ，是一个非常好用的高级语言模型框架。

### 主要软件准备

- **transformers**框架：

    pip安装：

    ```shell
    pip install transformers
    ```
    
- **PyTorch**：见 [PyTorch官网](https://pytorch.org)

- **Keras**：见 [Keras官网](https://keras.io)

### Pytorch-BERT中文预训练模型下载

包含三个文件：

|name | size 
|:------|:------
|config.json | 1KB |
|pytorch_model.bin | 392MB |
|bocab.txt | 107KB |

>其中 `pytorch_model.bin` 就是 `bert-base-chinese`:
>
>12-layer, 768-hidden, 12-heads, 110M parameters.
>Trained on cased Chinese Simplified and Traditional text.
>
>建议不要用transformers自带的命令下载，由于众所周知的原因是奇慢无比，而且容易断线。
>
>我传到了度盘上面，如果还嫌慢可以自己找地方下载。
>
>度盘下载地址：https://pan.baidu.com/s/1CCylS1nkL4ut8T3nr9cUNA 提取码：3ypf

### 硬件准备

- 支持CUDA运算的显卡（计算卡），最好给点力，要不然训练很久。

- 显存尽量大点，不然 batch size 小了训练很憋屈。

### 数据准备

我采用的是 THUCNews 数据集的**子集**，由清华NLP组提供。

这个数据集是针对新闻标题进行分类的数据集。可以在[这里](https://github.com/DrDavidS/Pytorch_Basic/tree/master/datasets/THUCNews)下载。文件以txt格式保存，打开以后可以看看内容。

简单介绍一下数据集，THUCNews是根据新浪新闻RSS订阅频道2005~2011年间的历史数据筛选过滤生成，包含74万篇新闻文档，划分出 14 个候选分类。

我们只采用了其中 10 个子类，包括

```
finance
realty
stocks
education
science
society
politics
sports
game
entertainment
```

训练数据共 180000 条，保存在 `train.txt` 中。测试数据保存在 `dev.txt` 和 `text.txt` 中，每个文件10000条。

数据格式如下：

```
《非诚勿扰》“冯女郎”车晓带妈妈闯世界(图)	9
美弗吉尼亚大学访华太设计签实习基地协议（组图）	1
华中科技大学2010年考研成绩查询开通	3
陈小艺“激吻照”疑似炒作	9
90岁老太半世纪撮合200多对新人(图)	5
袁立挑选钻戒被疑婚期将近 男伴酷似梁朝伟(图)	9
国务院：严打拐卖操控未成年人违法犯罪	6
郎平不惧土耳其劲旅窘境：我就喜欢接这种烂摊子	7
...
```

其中末尾数字代表标签类型，数字和种类对照参见 `class.txt`。数字标签和文本中间用制表符隔开。

## 预先准备

### 导入必要包

In [51]:
import gc
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.preprocessing.sequence import pad_sequences  # padding句子用
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification

# 如果使用jupyter lab，那么打开以下插件，并且安装了node.js
# 参考：https://stackoverflow.com/questions/57343134/jupyter-notebooks-not-displaying-progress-bars
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager@2

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
An error occured.
ValueError: "@jupyter-widgets/jupyterlab-manager@2" is not a valid npm package
See the log file for details:  /tmp/jupyterlab-debug-tj_n2lgq.log


In [2]:
print(f"PyTorch 版本： {torch.__version__}")
print(f"Transformers 版本： {transformers.__version__}")

PyTorch 版本： 1.6.0
Transformers 版本： 3.0.2


注意我用的 PyTorch 版本是 1.6.0，用新不用旧。

理论上这些代码在 PyTorch 1.3.1 以上都可以运行，如果有问题请在 Github 上向我提问。

### GPU

检查GPU状态，我之前用的是 Tesla M40，不过现在只有一块可怜的 GTX1070 了。

In [3]:
# GPUcheck

print("CUDA Available: ", torch.cuda.is_available())
n_gpu = torch.cuda.device_count()

if torch.cuda.is_available():
    print("GPU numbers: ", n_gpu)
    print("device_name: ", torch.cuda.get_device_name(0))
    device = torch.device("cuda:0")  # 注意选择
    torch.cuda.set_device(0) 
else :
    device = torch.device("cpu")
    

print(f"当前设备编号：{torch.cuda.current_device()}")

CUDA Available:  True
GPU numbers:  1
device_name:  GeForce GTX 1070
当前设备编号：0


## BERT相关设置

为了规范和工程化，我们最好不要把一些超参数零零散散地分散在整个代码中。我们需要使用一个`Config()`类来统一管理这些参数。

In [4]:
class Config(object):
    """配置参数"""
    def __init__(self):
        self.model_name = 'Bert_NEWS_CLF.bin'
        self.bert_path = './bert-chinese/'
        self.train_file = '../datasets/THUCNews/train.txt'
        
        self.num_classes = 10                    # 类别数(按需修改)
        self.hidden_size = 768                   # 隐藏层输出维度
        self.hidden_dropout_prob = 0.1           # dropout比例
        self.batch_size = 128                    # mini-batch大小
        self.max_len = 32                        # 句子的最长padding长度
        
        self.epochs = 3                          # epoch数
        self.learning_rate = 2e-5                # 学习率        

        self.save_path = './saved_model/'        # 模型训练结果保存路径
        
        # self.fp16 = False
        # self.fp16_opt_level = 'O1'
        # self.gradient_accumulation_steps = 1
        # self.warmup_ratio = 0.06
        # self.warmup_steps = 0
        # self.max_grad_norm = 1.0
        # self.adam_epsilon = 1e-8
        # self.class_list = class_list                              # 类别名单
        # self.require_improvement = 1000                                # 若超过1000batch效果还没提升，则提前结束训练
        
config = Config()

## 数据处理

### 读取数据

放在 `./data/train.txt`中，有需要请自己改路径。

In [5]:
file = config.train_file

with open(file, encoding="utf-8") as f:
    sentences_and_labels = [line for line in f.readlines()]
f.close()

In [6]:
# 前几句
sentences_and_labels[0:10]

['中华女子学院：本科层次仅1专业招男生\t3\n',
 '两天价网站背后重重迷雾：做个网站究竟要多少钱\t4\n',
 '东5环海棠公社230-290平2居准现房98折优惠\t1\n',
 '卡佩罗：告诉你德国脚生猛的原因 不希望英德战踢点球\t7\n',
 '82岁老太为学生做饭扫地44年获授港大荣誉院士\t5\n',
 '记者回访地震中可乐男孩：将受邀赴美国参观\t5\n',
 '冯德伦徐若瑄隔空传情 默认其是女友\t9\n',
 '传郭晶晶欲落户香港战伦敦奥运 装修别墅当婚房\t1\n',
 '《赤壁OL》攻城战诸侯战硝烟又起\t8\n',
 '“手机钱包”亮相科博会\t4\n']

数据以 `table` 分割，所以用 `split('\t')`：

In [7]:
seq, label = sentences_and_labels[2].split('\t')
print(seq)
print(label)

东5环海棠公社230-290平2居准现房98折优惠
1



In [8]:
sentences = []
labels = []

for sentence_with_label in sentences_and_labels:
    sentence, label = sentence_with_label.split('\t')
    sentences.append(sentence)
    labels.append(label)

In [9]:
print(sentences[0:5])
print(labels[0:5])

['中华女子学院：本科层次仅1专业招男生', '两天价网站背后重重迷雾：做个网站究竟要多少钱', '东5环海棠公社230-290平2居准现房98折优惠', '卡佩罗：告诉你德国脚生猛的原因 不希望英德战踢点球', '82岁老太为学生做饭扫地44年获授港大荣誉院士']
['3\n', '4\n', '1\n', '7\n', '5\n']


### 按字拆分：
    
    
    
按照BERT的要求，我们需要使用输入为：
```
[CLS]<句子A>[SEP]<句子B>[SEP]
```
这样的形式，但是很明显我们的新闻标题是不适合这样分开的，所以我们的输入形式是：
```
[CLS]<句子A>[SEP]
```

BERT不需要分词，我们只要直接将他们转换为 `vocab.txt` 字典中对应的字索引即可，包括`[CLS]`和`[SEP]`。

例如：

In [10]:
tokenizer = BertTokenizer.from_pretrained(config.bert_path, 
                                          do_lower_case=True)
tokenizer

其中，`BertTokenizer.from_pretrained()` 是从本地（或者在线下载）相关的BertToken文件。

我本地已经有此文件了，所以地址就存在了 `config.bert_path` 中，直接指定即可导入。

In [11]:
# 旧版代码：
tokenized_texts = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]
# 新版代码：
# text_batch = sentences
# encoding = tokenizer(text_batch, 
#                     return_tensors='pt',  # pt 指 pytorch，tf 就是 tensorflow 
#                     padding=True,  # padding到最长的那句话
#                     truncation=True,  # 激活并控制截断
#                     max_length=50)
#input_ids = encoding['input_ids']
#input_ids

`add_special_tokens=True` 指的是是否要在句首和句尾自动添加 `[CLS]` 和 `[SEP]` token。

In [12]:
# 这句话的input_ids
print(f"Tokenize 前的第一句话：\n{sentences[0]}\n")
print(f"Tokenize 后的第一句话: \n{tokenized_texts[0]}")

Tokenize 前的第一句话：
中华女子学院：本科层次仅1专业招男生

Tokenize 后的第一句话: 
[101, 704, 1290, 1957, 2094, 2110, 7368, 8038, 3315, 4906, 2231, 3613, 788, 122, 683, 689, 2875, 4511, 4495, 102]


以上面的例子说明：

首先encode包含了两个动作，

**第一**，对句子

    ```中华女子学院：本科层次仅1专业招男生```

的前后添加标签，即：

    ```[CLS]中华女子学院：本科层次仅1专业招男生[SEP]```

**第二**，将添加标签后的句子按字符（标点符号也单独算一个字符）分开，然后转换为 `input_ids`：

    ```[101, 704, 1290, 1957, 2094, 2110, 7368, 8038, 3315, 4906, 2231, 3613, 788, 122, 683, 689, 2875, 4511, 4495, 102]```

> 需要说明的是，其中 `101` 是 `[CLS]` 的索引，`102` 是 `[SEP]` 的索引。

上述过程称为 `tokenized`。

In [13]:
print (len(tokenized_texts))  # 180000句话

180000


### Padding

为了保证输入长度的统一，我们需要对句子进行padding。

本例中采用的是新闻标题，所以标题不会太长，我们限定为 `32` 个字符，此参数在 `config.max_len` 中。

一旦标题长度超过32个字符，则会截断超过部分不用；如果不足 `32` 个字符，则执行 `pad_sequences` （即`padding`）操作。

In [14]:
# 输入padding
# 此函数在keras里面
input_ids = pad_sequences([txt for txt in tokenized_texts],
                          maxlen=config.max_len, 
                          dtype="long", 
                          truncating="post", 
                          padding="post")

In [15]:
print(f"Tokenize 前的第一句话：\n\n{sentences[0]}\n\n")
print(f"Tokenize 后的第一句话: \n\n{tokenized_texts[0]}\n\n")
print(f"Padding 后的第一句话： \n\n{input_ids[0]}")

Tokenize 前的第一句话：

中华女子学院：本科层次仅1专业招男生


Tokenize 后的第一句话: 

[101, 704, 1290, 1957, 2094, 2110, 7368, 8038, 3315, 4906, 2231, 3613, 788, 122, 683, 689, 2875, 4511, 4495, 102]


Padding 后的第一句话： 

[ 101  704 1290 1957 2094 2110 7368 8038 3315 4906 2231 3613  788  122
  683  689 2875 4511 4495  102    0    0    0    0    0    0    0    0
    0    0    0    0]


其实 `padding` 之后还可转换回来，

很容易看出每个字，包括`[PAD]`、`[CLS]`、`[SEP]`所在的位置:

In [16]:
# 转换回来
raw_texts = [tokenizer.decode(input_ids[0])]
print(raw_texts)
print(len(raw_texts))

['[CLS] 中 华 女 子 学 院 ： 本 科 层 次 仅 1 专 业 招 男 生 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']
1


## BERT的输入准备

### 注意力mask（attention masks）：

BERT 模型的核心是 Transformer 结构，其中很重要的一点就是 self-attention 结构。

BERT-Chinese 模型同 BERT-base 模型结构一致，每层有12个自注意头，为了不让这些 self-attention 结构注意到补全的`[PAD]`部分，我们需要输入一个 attention_masks 标签，告诉模型哪些内容是真实内容，哪些是无意义的`[PAD]`。

刚刚说到被 `padding` 部分是不需要被 attention 到的。相当于这部分在  attention_masks 中的标签就是真实句子为 1 ，padding 部分为 0 。所以我们得到 attention masks：

In [17]:
# 创建attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

In [18]:
# 第一句话的 attention_masks
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


###  准备Labels

首先准备Labels。这些标题的 Labels 在一开始就已经分离开来，保存在了 `labels` 里面
 
这里可以用 `train_test_split` 来分。注意，由于多了一个 `attention_masks` 所以我们需要用两次 `train_test_split`，并且采用相同的随机种子。

In [19]:
print(len(labels))
print(labels[0:10])

180000
['3\n', '4\n', '1\n', '7\n', '5\n', '5\n', '9\n', '1\n', '8\n', '4\n']


由于现在的labels里面并不是数字，而且有换行符`\n`，我们需要一些处理：

In [20]:
clean_labels = []
for label in labels:
    clean_labels.append(int(label.strip('\n')))

print(clean_labels[0:10])

[3, 4, 1, 7, 5, 5, 9, 1, 8, 4]


In [21]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, clean_labels, 
                                                            random_state=2019, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2019, test_size=0.1)

In [22]:
train_labels[0:10]

[4, 3, 4, 3, 8, 8, 1, 8, 7, 7]

In [23]:
print(f"      标签总数：", len(labels))
print(f"训练集标签总数：", len(train_labels))
print(f"验证集标签总数：", len(validation_labels))

      标签总数： 180000
训练集标签总数： 162000
验证集标签总数： 18000


和 TF2.0 可以直接接受`ndarray`不同，PyTorch要求我们先将数据转化为 `Tensor` 形式再输入模型。

现所以我们准备 Tensor 化。

> 由于直接使用 `torch.tensor()` 会弹出警告，所以我们使用更安全的代码。

**旧**：
```python
train_inputs = torch.tensor(train_inputs)
```

**新**:
```python
train_inputs = torch.tensor(train_inputs).clone().detach()
```

In [24]:
# tensor化
train_inputs = torch.tensor(train_inputs).clone().detach()
validation_inputs = torch.tensor(validation_inputs).clone().detach()
train_labels = torch.tensor(train_labels).clone().detach()
validation_labels = torch.tensor(validation_labels).clone().detach()
train_masks = torch.tensor(train_masks).clone().detach()
validation_masks = torch.tensor(validation_masks).clone().detach()

In [25]:
print(len(validation_inputs))
print(len(validation_labels))
print(len(validation_masks))

18000
18000
18000


### 创建迭代器

我们采用

`torch.utils.data.TensorDataset` 将他们封装为 `TensorDataset` 的形式，

`torch.utils.data.RandomSampler` 采用随机采样的方法从中采样，

`torch.utils.data.DataLoader` 自动形成迭代器。

> 注意 batch_size 的设置大小和显存大小密切相关！

In [26]:
# batch size
print(config.batch_size)

128


In [27]:
# 形成训练数据集
train_data = TensorDataset(train_inputs, train_masks, train_labels)  
# 随机采样
train_sampler = RandomSampler(train_data) 
# 读取数据
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=config.batch_size)


# 形成验证数据集
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
# 随机采样
validation_sampler = SequentialSampler(validation_data)
# 读取数据
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=config.batch_size)

## BERT的微调

在准备好输入以后，现在我们开始微调BERT模型。

使用 `BertForSequenceClassification`，它就是一个普通BERT模型，在最后面加了一个线形层用于分类。

### 导入模型

直接使用 `from_pretrained` 导入预训练好的中文 BERT 模型：

In [28]:
# 统计标签种类
label_count = len(set(labels))
print(label_count)

config.num_classes = label_count

10


In [29]:
# 读取 BertForSequenceClassification 模型，
# 是一个预训练的BERT模型，在最后面加了一个线形层用于分类。

model = BertForSequenceClassification.from_pretrained(config.bert_path, 
                                                      num_labels=config.num_classes)
model.cuda()

# 注意：
# 在新版的 Transformers 中会给出警告
# 原因是我们导入的预训练参数权重是不包含模型最终的线性层权重的
# 不过我们本来就是要“微调”它，所以这个情况是符合期望的

Some weights of the model checkpoint at ./bert-chinese/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at .

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

在完成 `model.cuda()` 这一步以后，模型就放在显存中了。同时也打印出了模型的具体结构。

> 需要说明的是，为了方便起见，我们直接采用了 `BertForSequenceClassification.from_pretrained()`，实际上如果有更灵活的需求时，应该使用 `BertModel.from_pretrained()` 搭建模型。两者的区别可以参考源代码。

例如，接下来给出一段魔改模型参考，我们采用了 `BERT` 输出后拼接其他特征再过一个线性层：

```python
class ModelBert(nn.Module):
    """
    新增 行业ID 特征拼接。
    
    我们想要对商品名称进行分类，商品名称是文本形式，会进入BERT模型。
    行业ID，即代码中的“HYID”本身就是数字形式，没必要放入BERT模型中，于是我们将 BERT 输出后的 768 维向量拼接
    tensor(HYID)，也就是变成了 769 维，再过一个 线形层 + softmax 输出分类结果。
    """
    def __init__(self, config):
        super(ModelBert, self).__init__()
        self.num_labels = config.num_classes
        self.bert = BertModel.from_pretrained(config.bert_path)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size + 1, config.num_classes)  # 这里维度 +1 放入HYID

    def forward(self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        HYID=None):
        
        outputs = self.bert(
                            input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds,)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        HYID_f = HYID.float()
        pooled_output = torch.cat((pooled_output, HYID_f), dim=1)  # 拼接在这里
        logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs        
        return outputs  # (loss), logits, (hidden_states), (attentions) 
    
    model = ModelBert(config)
    model.cuda()
```

### 准备微调

众所周知，我们所使用的 BERT 是一个预训练好的模型，我们需要根据不同的任务，比如这里的 文本分类，或者 NER 之类的任务，使用本地数据进行微调。

首先我们设置好相关的优化器和评价函数。

In [30]:
# BERT fine-tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']

# 权重衰减
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
     'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
     'weight_decay': 0.0}]

其中，`no_decay`见[issue#492](https://github.com/huggingface/transformers/issues/492)

In [31]:
# 优化器
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=config.learning_rate)

In [32]:
# 准确率计算函数
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [33]:
# 保存loss
train_loss_set = []
# epochs 
print(f"训练 Epochs： {config.epochs}")

训练 Epochs： 3


### 开始微调

3个epoch：

In [34]:
# BERT training loop
for _ in range(config.epochs): 
    ## 训练
    print(f"当前epoch： {_}")
    # 开启训练模式
    model.train()
    tr_loss = 0  # train loss
    nb_tr_examples, nb_tr_steps = 0, 0
    # Train the data for one epoch
    for step, batch in tqdm(enumerate(train_dataloader)):
        # 把batch放入GPU
        batch = tuple(t.to(device) for t in batch)
        # 解包batch
        b_input_ids, b_input_mask, b_labels = batch
        # 梯度归零
        optimizer.zero_grad()
        # 前向传播loss计算
        output = model(input_ids=b_input_ids, 
                       attention_mask=b_input_mask, 
                       labels=b_labels)  # 有labels的时候，且labels>1就直接返回Cross-Entropy
        loss = output[0]
        # print(loss)
        # 反向传播
        loss.backward()
        # Update parameters and take a step using the computed gradient
        # 更新模型参数
        optimizer.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
    print(f"当前 epoch 的 Train loss: {tr_loss/nb_tr_steps}")

当前epoch： 0



当前 epoch 的 Train loss: 0.29743796679386975
当前epoch： 1



当前 epoch 的 Train loss: 0.1526083102741089
当前epoch： 2



当前 epoch 的 Train loss: 0.10776557406413517


### 批量验证

我们之前已经使用 `train_test_split()` 为交叉验证做好了准备。因此可以直接进入验证：

In [35]:
# 验证状态
model.eval()

# 建立变量
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
# Evaluate data for one epoch

In [38]:
# 验证集的读取也要batch
for batch in tqdm(validation_dataloader):
    # 元组打包放进GPU
    batch = tuple(t.to(device) for t in batch)
    # 解开元组
    b_input_ids, b_input_mask, b_labels = batch
    # 预测
    with torch.no_grad():
        # segment embeddings，如果没有就是全0，表示单句
        # position embeddings，[0,句子长度-1]
        logits = model(input_ids=b_input_ids, 
                       attention_mask=b_input_mask,
                       token_type_ids=None,
                       position_ids=None)  
                       
    # print(logits[0])
    # Move logits and labels to CPU
    logits = logits[0].detach().cpu().numpy()  # 注意这里的logits是在softmax之前，所以和不为1
    label_ids = b_labels.to('cpu').numpy()
    # print(logits, label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)  # 计算准确率
    eval_accuracy += tmp_eval_accuracy  # 准确率积累
    nb_eval_steps += 1  # 步数积累
print(f"Validation Accuracy: {eval_accuracy/nb_eval_steps}")    


Validation Accuracy: 0.9465093085106382


## 单条预测

为了详细说明我们微调后的 BERT 预测过程，我们使用单条预测的方式展示一下相关结果。

有这样一条新闻标题：

```词汇阅读是关键 08年考研暑期英语复习全指南```

其类别为

```3. education```

现在我们采用单条预测的形式看看模型对它的预测。

流程和之前一样，也是先 tokenize 再做 padding 和 mask。具体如下：

In [40]:
test_sent = "词汇阅读是关键 08年考研暑期英语复习全指南"

In [41]:
test_tokenized_texts = [tokenizer.encode(test_sent, add_special_tokens=True)]

In [42]:
test_input_ids = pad_sequences([txt for txt in test_tokenized_texts],
                          maxlen=config.max_len, 
                          dtype="long", 
                          truncating="post", 
                          padding="post")

In [43]:
print(f"Tokenize 前的第一句话：\n{test_sent}\n\n")
print(f"Tokenize 后的第一句话: \n{test_tokenized_texts}\n\n")
print(f"Padding 后的第一句话： \n{test_input_ids}")

Tokenize 前的第一句话：
词汇阅读是关键 08年考研暑期英语复习全指南


Tokenize 后的第一句话: 
[[101, 6404, 3726, 7325, 6438, 3221, 1068, 7241, 8142, 2399, 5440, 4777, 3264, 3309, 5739, 6427, 1908, 739, 1059, 2900, 1298, 102]]


Padding 后的第一句话： 
[[ 101 6404 3726 7325 6438 3221 1068 7241 8142 2399 5440 4777 3264 3309
  5739 6427 1908  739 1059 2900 1298  102    0    0    0    0    0    0
     0    0    0    0]]


In [44]:
# 创建attention masks
test_attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
    seq_mask = [float(i > 0) for i in seq]
    test_attention_masks.append(seq_mask)

In [45]:
# tensor化
test_inputs = torch.tensor(test_input_ids).clone().detach()
test_masks = torch.tensor(test_attention_masks).clone().detach()

In [46]:
# 形成验证数据集
# 为了通用，这里还是用了 DataLoader 的形式
test_data = TensorDataset(test_inputs, test_masks)
# 随机采样
test_sampler = SequentialSampler(test_data)
# 读取数据
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=config.batch_size)

In [47]:
# 验证集的读取也要batch
model.eval()

for batch in tqdm(test_dataloader):
    # 元组打包放进GPU
    batch = tuple(t.to(device) for t in batch)
    # 解开元组
    b_input_ids, b_input_mask = batch
    # 预测
    outputs = model(input_ids=b_input_ids, 
                    attention_mask=b_input_mask, 
                    token_type_ids=None,
                    position_ids=None)
                       
    # Move logits and labels to CPU
    scores = outputs[0].detach().cpu().numpy()  # 每个字的标签的概率
    pred_flat = np.argmax(scores, axis=1).flatten()
    # label_ids = b_labels.to('cpu').numpy()  # 真实labels
    print(pred_flat)  # 预测值

[3]



可以看出，预测是正确的。这样的单句预测常常用在在需要演示的情况下，这时候直接替换 `test_sent` 就行了。

如果为了方便起见，还可以打包为一个函数。

## 保存和读取模型

模型训练完毕后，我们需要对模型进行保存。 PyTorch 的保存和 Transformers 是不太一样的，这里需要**特别说明**。

### 基于 PyTorch 保存 / 读取

一般我们采用 PyTorch 自带的模型保存方法。此方法可以保存读取各种网络结构的模型。

使用 `torch.save()` 方法将模型参数保存为 `state_dict`，读取时候也是相同的道理。

In [60]:
# 创建文件夹
if not os.path.exists(config.save_path):
    os.makedirs(config.save_path)
    print("文件夹不存在，创建文件夹!")
else:
    pass

文件夹不存在，创建文件夹


In [61]:
# 保存模型
model_to_save = model.module if hasattr(model, 'module') else model                          # 用于多卡训练的情况
torch.save(model_to_save.state_dict(), os.path.join(config.save_path, config.model_name))    # 模型保存

保存完毕后我们读取模型。

> 注意，这里我分别读取了 **模型结构** 和 **Tokenizer**，两者路径要按实际调整。

In [66]:
# 读取模型
tokenizer = BertTokenizer.from_pretrained(config.bert_path)
model = BertForSequenceClassification.from_pretrained(config.bert_path, num_labels=config.num_classes)
model.load_state_dict(torch.load(os.path.join(config.save_path, config.model_name)))

model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

同样的，如果我要读取魔改的模型，比如前面提到的 `ModelBert`，一样可采用 `model.load_state_dict()` 来读取，但是先要定义好模型结构才行。

示例如下，假设已经搭建好了模型结构了，现在需要读取之前训练好且保存好的模型参数：

```python
tokenizer = BertTokenizer.from_pretrained(config.bert_path)
model = ModelBert(config)
model.load_state_dict(torch.load(os.path.join(config.save_path, config.model_name)))
```

### 基于 Transformers 保存 / 读取（不推荐）

如果直接采用的是 Transformers 自带的模型结构，比如上文的 `BertForSequenceClassification`，Transformers 自带了一种 `save_pretrained` 保存方法，可以方便地将模型保存为二进制文件。

> 为什么不推荐这个方法呢？
>
> 因为如果你需要以 BERT 为核心，对模型进行魔改的话，这个功能很可能由于模型结构的不同而报错或者导致效果一塌糊涂。
>
> 魔改模型参考上文中的 `ModelBert` 部分，换句话说，模型结构由自己定义，已经和 Transformers 自带的结构不一致了。

In [ ]:
# 创建文件夹
if not os.path.exists(config.save_path):
    os.makedirs(config.save_path)
    print("文件夹不存在，创建文件夹!")
else:
    pass

# 保存
model_to_save2 = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save2.save_pretrained(config.save_path)
tokenizer.save_pretrained(config.save_path)

In [ ]:
# 读取模型
model = BertModel.from_pretrained(config.save_path)
tokenizer = BertTokenizer.from_pretrained(config.save_path)

### 小结

当前我采用的卡是 Tesla M40，单卡24GB显存。

总共跑了3个 epoch，batch_size 是 512。

`tqdm_notebook` 的输出时间有点迷，这里就不记录了。之前用 `tqdm` 的时候大概一个 epoch 在 23 分钟左右。

测试集仅仅是前向传播，总共18000条数据只花了不到一分钟就完成了。

最后准确率94.4%。

实际上这个准确率有很大的提升空间，包括调参，包括清洗什么的都没有做（实际上数据集很干净了也没必要做），直接就是BERT梭哈。

> 总体来说，我写的这个教程还是很详细很友好的，而且没有使用老旧的 `pytorch-pretrained-bert`，而是当前最新版本的 `transformers`。
>
> 其次，每一步几乎都有比较详细的中间过程解释，我相信稍微熟悉 PyTorch 和 BERT 基本结构的人都能快速上手。
>
> 总的来说，我觉得比什么 CSDN 或者 知乎 上面零零散散的代码及解释强多了...
>
> 有什么问题可以在我的 Github 上面提出来，地址在这里：[DrDavidS](https://github.com/DrDavidS/basic_Machine_Learning) 。

### 后续改进方向

还有很多工作待补充：

- Apex.fp16改写
- Scheduler设置
- Early-Stop设置